# JuMP.jl: Beyond the Basics
In the first part of our `JuMP.jl` introduction, we learned how to model/solve simple optimization problems with scalar decision variables. In this part, we take a deeper dive into `JuMP.jl`'s features and to have the tools we need to tackle real problems.

## Resources
Again, for reference, here are resources to learn more and get help:
- The tutorials, examples, manuals, and guides in `JuMP.jl`'s documentation: https://jump.dev/JuMP.jl/stable/
- The Julia optimization forum: https://discourse.julialang.org/c/domain/opt/13
- Julia Programming for Operations Research 2/e (not always up-to-date): https://www.softcover.io/read/7b8eb7d0/juliabook2/introduction

## Models and Solvers
We have learned how to create `Model` objects with an optimizer using its default settings. Now let's take a closer look.

### Solver Specification
As before, we can create by passing an optimizer:

In [ ]:
using JuMP, HiGHS

model = Model(HiGHS.Optimizer)

Alternatively, we can create the `Model` add the optimizer later (any time before calling `optimize!`) via `set_optimizer`:

In [ ]:
model = Model()
set_optimizer(model, HiGHS.Optimizer)

This all allows us to create a `Model` object and attach a solver to it. However, where possible the optimizer should be provided directly to the `Model` object for better error messaging (e.g., adding a constraint that the solver doesn't support). 

Often times we may wish to specify options (attributes) to the solver. One way to accomplish this is via `optimizer_with_attributes`: 

In [ ]:
model = Model(optimizer_with_attributes(HiGHS.Optimizer, "output_flag" => false, "presolve" => "on"))

Here the attributes are solver specific and can be found by checking the documentation associated with each solver. We can also specify/modify attributes using `set_optimizer_attribute`:

In [ ]:
model = Model(HiGHS.Optimizer)
set_optimizer_attribute(model, "output_flag", false)
set_optimizer_attribute(model, "presolve", "on")

For convenience, `JuMP.jl` provides a few solver-agnostic methods for setting common attributes such as turning the output off and setting a time limit:

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model) # turn the output printing off
set_time_limit_sec(model, 60.0) # set a time limit

### File Writing/Reading
`JuMP.jl` does support writing models to files via `write_to_file` and creating a model from a file via `read_from_file`:

In [ ]:
write_to_file(model, "model.mps")
read_model = read_from_file("model.mps")

For more information on supported files and details about this refer to https://jump.dev/JuMP.jl/stable/manual/models/#Write-a-model-to-file.

### `MathOptInterface.jl` Backends and Performance
More advanced users may wish to better understand what is going on behind the scenes and squeeze out some better performance. This section will touch upon some of these considerations at a surface level. A more throughout discussion is provided at https://jump.dev/JuMP.jl/stable/manual/models/#Backends

`JuMP.jl` `Model`s are thin wrappers `MathOptInterface.jl` models which are what actually store the optimization problem and interface to the solvers. These model(s) employed by `MathOptInterface.jl` behind the scenes are called the backend and can be accessed via `backend`. Let's take a look:

In [ ]:
model = Model(HiGHS.Optimizer)
b = backend(model)

There is quite a bit going on here in addition to the `MOI` `HiGHS.Optimizer` model. We'll briefly describe these different layers. 

The `MOIU.CachingOptimizer` is a layer that allows us to build models incrementally (e.g., adding variables one by one) even if the solver doesn't support that. All the information is stored in `MOIU.UniversalFallback{MOIU.Model{Float64}}` which acts as a cache:

In [ ]:
b.model_cache


The cache model can always be build/modified incrementally. This is then used to efficiently update the optimizer model `MOIB.LazyBridgeOptimizer{HiGHS.Optimizer}` when appropriate:

In [ ]:
b.optimizer

Notice the optimizer model (`HiGHS.Optimizer`) is wrapped in `MOIB.LazyBridgeOptimizer`. The bridge layer allows constraints to be transformed (i.e., bridged) to forms that a solver supports. For example, we might need to split an interval constraint into 2 inequality constraints.

If constraint bridges are unnecessary for our model, and we wish to decrease the start-up latency, we can use `add_bridges = false` when we create the `JuMP.jl` `Model` to eliminate this layer:

In [ ]:
model = Model(HiGHS.Optimizer; add_bridges = false)
backend(model)

Notice that the `MOIB.LazyBridgeOptimizer` layer on the optimizer is now gone. 

To take this a step further, we can eliminate the cache model entirely by creating a model via `direct_model`:

In [ ]:
model = direct_model(HiGHS.Optimizer())
backend(model)

Now we only have the `HiGHS.Optimizer` as the backend model. This avoids the overhead of the caching model which effectively creates two copies of our model. However, there are a few caveats:
- This will not work for solvers that cannot be build incrementally (e.g., Ipopt)
- There are no bridges to reformulate constraints into acceptable forms
- The behavior of querying the solution after modification to the problem is solver specific

So direct models can help increase performance, but should be used carefully.

## Variables
Let's take a deeper dive into more of the things we can do with `@variable`.

### Containers and Sets
We have already seen how to add individual scalar variables, now let's see how to add multiple variables at once.

`JuMP.jl` uses 3 data structures to store variable collections:
- `Array`s: The native Julia arrays
- `DenseAxisArray`s: Dense arrays with arbitrary indices
- `SparseAxisArray`s: Sparse arrays with arbitrary indices

Arrays are created using integer indices of the form `1:n`. For instance, the matrix:

In [ ]:
model = Model()
@variable(model, a[1:2, 1:4])

This creates a 2 x 4 matrix of variables that is stored to `a` which we can index and use in defining our problem.

We can also create an n-dimensional vector variable $x \in \mathbb{R}^n$ with upper and lower bounds:

In [ ]:
n = 5
l = [1, 2, 3, 4, 5]
u = [10, 11, 12, 13, 14]

@variable(model, l[i] <= x[i = 1:n] <= u[i])

Notice we declare an index `i` to help us define the appropriate values. 

We can use other index forms that don't conform to `1:n` and make `DenseAxisArray`s:

In [ ]:
@variable(model, z[i = 2:3, j = 1:2:3] >= i + 2j)

We don't even have to use integers:

In [ ]:
@variable(model, w[["red", "blue"], 1:5] <= 1)

For indices that do not form a rectangular set, a `SparseAxisArray` is formed:

In [ ]:
@variable(model, u[i = 1:2, j = i:3])

We can even add a conditional statement after a `;` when defining indices:

In [ ]:
@variable(model, v[i = 1:3, j = 1:4; i + j <= 4])

### Integrality
To specify integer variables, we need only add the `Int` argument:

In [ ]:
@variable(model, integer_x, Int)

or by setting `integer = true`:

In [ ]:
@variable(model, integer_z, integer = true)

Similarly, we create binary variables via the `Bin` argument:

In [ ]:
@variable(model, binary_x, Bin)

or by using `binary = true`:

In [ ]:
@variable(model, binary_z, binary = true)

### Exercise: Nodal Variables
**Problem**
- Create a variable named `xp`
- `xp` should be integer valued between 0 and 3
- `xp` should be indexed over each arc `(i, j)` in `arcs`

In [ ]:
arcs = [(1, 2), (1, 3), (3, 2), (2, 4)]

# PUT CODE HERE


### Other Options
There are a variety of other things we can do with variables. We can create a fixed variable:

In [ ]:
@variable(model, x_fixed == 42)

We can specify the initial guess to pass on to the solver via `start`:

In [ ]:
@variable(model, q, start = 2)

We can also specify lower/upper bounds via the keyword arguments `lower_bound` and `upper_bound`.

### Anonymous Variables
Normally when we create variables they are registered in the model, so we can access them via `model[:var_name]`. For instance, consider a typical definition of a scalar variable:

In [ ]:
model = Model()
@variable(model, x >= 0)

This does a few things:
- Add a scalar variable with a name `"x"` to `model`
- Add a lower bound of 0
- Create a Julia variable `x` that stores a reference to the optimization variable
- Registers that Julia variable in `model` such that is can be accessed via `model[:x]`

In [ ]:
@show x 
@show model[:x];

However, this registration means we cannot add another variable with the same name:

In [ ]:
@variable(model, x == 0)

If this behavior is prohibitive, we can define an anonymous variable:

In [ ]:
x = @variable(model, lower_bound = 0, base_name = "x")

This adds an optimization variable to the model with a lower bound and name `"x"`, but doesn't register it. We store the resulting variable reference in the Julia variable `x`.

### Modify Variables
There are a variety of ways to change variables after they are created. Some common methods include:
- `set_lower_bound`
- `set_upper_bound`
- `fix`
- `set_start_value`
- `set_binary`
- `set_integer`
- `delete`

For example:

In [ ]:
set_upper_bound(x, 10)
set_integer(x)
delete(model, x)

There are many more things we can do, see https://jump.dev/JuMP.jl/stable/manual/variables/ to learn more. 

## Expressions
Sometimes we may want to use a mathematical expression in multiple constraints and/or the objective. We can create expressions using `@expression`. To motivate this, let's create a model with variables:

In [ ]:
model = Model()
@variable(model, x[1:2]);

### Affine/Quadratic Expressions
We can create affine/quadratic expressions using `@expression`. For instance:

In [ ]:
my_expr = @expression(model, x[1]^2 - 3x[2])

creates an anonymous expression that we can use elsewhere. We can also create named/registered expressions by adding a name argument:

In [ ]:
@expression(model, my_expr, x[1]^2 - 3x[2])
@show my_expr;

This creates a Julia variable `my_expr` to access the expression and registers it, so we can get it by indexing `model`:

In [ ]:
model[:my_expr]

We can also create a container of expressions, just like we can for variables:

In [ ]:
@expression(model, expr[i = 1:2], 4x[i]^2)

### Linear Algebra
For linear/quadratic expressions, `JuMP.jl` supports linear algebra operations. For instance, consider $x^T A y$:

In [ ]:
@variable(model, y[1:3])
A = [1 2 4; 2 6 1]

@expression(model, x' * A * y)

which is equivalent to:

In [ ]:
@expression(model, sum(x[i] * A[i, j] * y[j] for j in 1:3, i in 1:2))

### Nonlinear Expressions
Any expressions that aren't linear/quadratic must be created using `@NLexpression`. For example:

In [ ]:
@NLexpression(model, nlexpr[i = 1:2], 2sin(x[i]))

The library of built-in univariate functions is derived those listed in https://github.com/JuliaMath/Calculus.jl/blob/master/src/differentiate.jl. 

If we want to use some other nonlinear function that is not natively supported, we can add our own! For instance, let's add the `logerfcx` from `SpecialFunctions.jl` using the `register` function:

In [ ]:
using SpecialFunctions

register(model, :logerfcx, 1, logerfcx, autodiff = true) # register a univariate function `logerfcx` and use auto differientiation for gradients
@NLexpression(model, [i = 1:2], logerfcx(x[i]))

For more information on nonlinear expressions see https://jump.dev/JuMP.jl/stable/manual/nlp/#Nonlinear-Modeling.

For performance reasons, `@expression` should be used when possible. This will make evaluating the hessian easier later on. Also, note that nonlinear expressions do not support linear algebra currently.

Development is currently underway to overhaul the entire nonlinear interface and remove the need for `@NLexpression` and its limitations. 

## Objectives
We have already seen how to set linear/quadratic objectives using `@objective`. Let's learn a little more.

### Modification
Once an objective is set, we can modify by simply calling `@objective` again:

In [ ]:
# Model to play with
model = Model()
@variable(model, x[1:2])

# Set objective
@objective(model, Min, 2x[1] + 3x[2])
@show objective_function(model)

# Change the objective
@objective(model, Min, 4x[1] + 3x[2])
@show objective_function(model);

If all we want to do is change a linear coefficient, then we can use `set_objective_coefficient` instead:

In [ ]:
set_objective_coefficient(model, x[1], 2)
objective_function(model)

### Nonlinear Objectives
In analogous manner to expressions, non-quadratic/affine objectives must be specified via `@NLobjective`:

In [ ]:
@NLobjective(model, Min, log(x[1]) + x[2]^2)

These incur the same limitations as `@NLexpression` and will be reworked with the current overhaul.

### Exercise: Linear Algebra Objective
**Problem**
- Create a quadratic objective
- The function is $x^T A y + b^T y + c^Tx$
- Maximize the objective

In [ ]:
A = [1 3 6; -9 2 1]
b = [3, -2, 0]
c = [2, 1]

# PUT CODE HERE


## Constraints
Previously, we saw how to add simple scalar constraints. We will now take a deeper dive into using constraints in `JuMP.jl`.

Let's setup a model and variables:

In [13]:
model = Model()
@variable(model, x[1:2])
@variable(model, y[1:3]);

### Using Sets/Containers
